# KoBERT finetuning

In [ ]:
# 파이썬 버전을 3.7.0으로 변경
!wget https://www.python.org/ftp/python/3.7.0/Python-3.7.0.tgz
!tar xvfz Python-3.7.0.tgz
!Python-3.7.0/configure
!make
!sudo make install

In [ ]:
!python --version

In [ ]:
# 필요 패키지 설치
!pip install mxnet
!pip install gluonnlp
!pip install pandas
!pip install tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install openpyxl

# /usr/local/lib/python3.10/dist-packages/mxnet/numpy/utils.py:37: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar. bool = onp.bool 해결
!pip install numpy==1.23.1

In [2]:
# KoBERT 깃허브에서 필요한 파일 불러오기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-u9hyke2g/kobert-tokenizer_4f2a86ffb26d4d9891077f0842a22ac6
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-u9hyke2g/kobert-tokenizer_4f2a86ffb26d4d9891077f0842a22ac6
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [3]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gluonnlp as nlp
import pandas as pd

In [4]:
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive
from sklearn.model_selection import train_test_split

In [5]:
# # CPU
# device = torch.device("cpu")

# GPU
device = torch.device("cuda:0")

In [6]:
# 구글 drive와 연동하여 학습에 이용할 파일 불러옴
drive.mount('/content/drive/')
data_input = pd.read_excel('/content/drive/MyDrive/junho/data/data_conversation.xlsx')

Mounted at /content/drive/


In [7]:
# 데이터를 제대로 불러왔는지 확인
data_input.sample(n=5)

,Text,Class
724846,그래,9
574763,여기 오류가 있는 것 같은데요,0
772205,이,9
995922,진짜야,8
90990,네 나는,9


In [8]:
# 읽어온 학습 데이터를 리스트로 변환
data_list = []
for t, c in zip(data_input['Text'], data_input['Class']):
  if type(t)==float:
    continue
  data = []
  data.append(t)
  data.append(str(c))
  data_list.append(data)

In [9]:
print(len(data_list))
print(data_list[0])
print(data_list[-1])

1043936
['안녕하세요 잘 들리시나요', '9']
['하면은 다 똑같은 간호산데 뭐', '1']


In [10]:
dataset_train, dataset_validation = train_test_split(data_list, test_size=0.01, random_state=1)

In [11]:
print(len(dataset_train))
print(len(dataset_validation))

print(dataset_train[-1])
print(dataset_validation[-1])

1033496
10440
['너가 자주 이용하는 인터넷 쇼핑몰은 뭔데', '11']
['아 좀 측정해', '11']


In [12]:
# 파라미터 정보
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
# 학습/테스트 데이터 전처리를 위한 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [14]:
# 데이터셋 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [15]:
# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_validation = BERTDataset(dataset_validation, 0, 1, tok, vocab, max_len, True, False)

In [16]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
validation_dataloader = torch.utils.data.DataLoader(data_validation, batch_size = batch_size, num_workers = 5)

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 19,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [19]:
# optimizer 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# schedule 설정
loss_fn = nn.CrossEntropyLoss() # 다중 분류를 위한 loss function
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
# 체크포인트 변수
checkpoint_path = '/content/drive/MyDrive/junho/checkpoint/'
checkpoint_file = 'emotion_classification_model_checkpoint.pt'

In [22]:
import os

# 체크포인트 저장 함수
def save_checkpoint(model, optimizer, epoch, path=checkpoint_path+checkpoint_file):
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
  }, path)
  print(f"체크포인트 저장 - epoch {epoch}")

# 체크포인트 로드 함수
def load_checkpoint(model, optimizer, path=checkpoint_path+checkpoint_file):
  if not os.path.isfile(path):
    return 0
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  print(f"체크포인트 로드 -  epoch {epoch}")
  return epoch

In [23]:
# 정확도 측정 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 학습 개시
start = load_checkpoint(model, optimizer)
for e in range(start, num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        # if batch_id % log_interval == 0:
        #     print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(validation_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    save_checkpoint(model, optimizer, e+1)

In [26]:
# 모델을 저장할 폴더 생성
path = '/content/drive/MyDrive/junho/models/'
if not os.path.isdir(path):
  os.mkdir(path)

os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/junho/models'

In [27]:
# 모델 이름
pt_name = 'emotion_classification_model.pt'
pkl_name = 'emotion_classification_model.pkl'

In [28]:
# 생성한 모델을 폴더에 저장
import joblib

torch.save(model, path + pt_name)
joblib.dump(model, path + pkl_name)

['/content/drive/MyDrive/junho/models/emotion_classification_model.pkl']

In [29]:
# 저장한 모델을 불러오기
pt_model = torch.load(path + pt_name)
pkl_model = joblib.load(path + pkl_name)

In [30]:
# 실수를 치역으로 한 가중치 값을 softmax함수를 사용하여 텍스트를 확률값으로 변환
def new_softmax(a) :
    c = np.max(a)         # 최댓값
    exp_a = np.exp(a-c)   # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)

In [31]:
# 번호에 따른 태그 출력 메서드
tags = [
    "약한 공포", "보통 공포", "강한 공포",
    "약한 슬픔", "보통 슬픔", "강한 슬픔",
    "약한 분노", "보통 분노", "강한 분노",
    "중립",
    "약한 기쁨", "보통 기쁨", "강한 기쁨",
    "약한 놀람", "보통 놀람", "강한 놀람",
    "약한 사랑", "보통 사랑", "강한 사랑"
  ]

In [34]:
def predict(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]
    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    pt_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = pt_model(token_ids, valid_length, segment_ids)
        test_eval=[]

        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            probability = []
            logits = np.round(new_softmax(logits), 3).tolist()
            for l, logit in enumerate(logits):
                probability.append(tags[l] + '(' + str(np.round(logit, 3)) + ')')
            probability.append(tags[np.argmax(logits)])
            print(probability)
    return probability

In [35]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    print(predict(sentence))
    print("\n")

하고싶은 말을 입력해주세요 : 와우


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


['약한 공포(0.009)', '보통 공포(0.796)', '강한 공포(0.003)', '약한 슬픔(0.042)', '보통 슬픔(2.53)', '강한 슬픔(0.009)', '약한 분노(0.038)', '보통 분노(2.913)', '강한 분노(0.016)', '중립(52.639)', '약한 기쁨(0.244)', '보통 기쁨(27.687)', '강한 기쁨(0.228)', '약한 놀람(0.044)', '보통 놀람(7.347)', '강한 놀람(0.018)', '약한 사랑(0.045)', '보통 사랑(5.365)', '강한 사랑(0.026)', '중립']
['약한 공포(0.009)', '보통 공포(0.796)', '강한 공포(0.003)', '약한 슬픔(0.042)', '보통 슬픔(2.53)', '강한 슬픔(0.009)', '약한 분노(0.038)', '보통 분노(2.913)', '강한 분노(0.016)', '중립(52.639)', '약한 기쁨(0.244)', '보통 기쁨(27.687)', '강한 기쁨(0.228)', '약한 놀람(0.044)', '보통 놀람(7.347)', '강한 놀람(0.018)', '약한 사랑(0.045)', '보통 사랑(5.365)', '강한 사랑(0.026)', '중립']


하고싶은 말을 입력해주세요 : 와우!
['약한 공포(0.166)', '보통 공포(1.038)', '강한 공포(0.154)', '약한 슬픔(1.109)', '보통 슬픔(3.662)', '강한 슬픔(0.317)', '약한 분노(1.211)', '보통 분노(3.726)', '강한 분노(0.34)', '중립(11.683)', '약한 기쁨(4.881)', '보통 기쁨(29.961)', '강한 기쁨(6.777)', '약한 놀람(1.003)', '보통 놀람(17.491)', '강한 놀람(1.003)', '약한 사랑(1.958)', '보통 사랑(12.583)', '강한 사랑(0.937)', '보통 기쁨']
['약한 공포(0.166)', '보통 공포(1.038)', '강한 공포(0.154